# SMILES-to-Vector Encoding for Drug Molecules

This notebook converts SMILES molecular representations to numerical vector encodings for the 1,028 unique drugs. We'll use molecular fingerprints and descriptors to create feature vectors suitable for machine learning.

## Dataset Overview
- Input: 1,028 unique drugs with SMILES representations
- Encoding methods: Morgan fingerprints, MACCS keys, and molecular descriptors
- Output: Parquet file with encoded molecular feature vectors

## Encoding Methods Used
1. **Morgan Fingerprints (ECFP)**: Circular fingerprints capturing local molecular environments
2. **MACCS Keys**: 166-bit structural keys for common molecular features
3. **Molecular Descriptors**: Physicochemical properties (MW, LogP, etc.)

## 1. Import Required Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# For SMILES processing and molecular descriptors
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, rdMolDescriptors
    from rdkit.Chem import AllChem
    from rdkit.Chem.AtomPairs import Pairs
    from rdkit.Chem import MACCSkeys
    print("RDKit imported successfully!")
except ImportError:
    print("RDKit not found. Will install it in the next step.")

# For data processing
from sklearn.preprocessing import StandardScaler
import os

print("Core libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

RDKit imported successfully!
Core libraries imported successfully!
Pandas version: 2.3.2
NumPy version: 2.3.0
Core libraries imported successfully!
Pandas version: 2.3.2
NumPy version: 2.3.0


## 2. Load Drug Data from Parquet

In [2]:
# Load the original dataset to get unique drugs with SMILES
print("Loading drug data...")

# First, try to load from the previously created one-hot encoded file
try:
    # Load the original dataset with SMILES
    df_original = pd.read_parquet('scope_onside_common_v3.parquet')
    
    # Extract unique drugs with SMILES
    unique_drugs_df = df_original.groupby('drug_chembl_id')['smiles'].first().reset_index()
    print(f"Loaded {len(unique_drugs_df)} unique drugs from original dataset")
    
except FileNotFoundError:
    print("Original dataset not found. Trying alternative approach...")
    # Alternative: Load from one-hot encoded file if available
    try:
        unique_drugs_df = pd.read_parquet('encoded_drugs_onehot.parquet')[['drug_chembl_id', 'smiles']]
        print(f"Loaded {len(unique_drugs_df)} unique drugs from encoded dataset")
    except FileNotFoundError:
        print("No dataset found. Please ensure the dataset files are available.")
        raise

print(f"\nDataset shape: {unique_drugs_df.shape}")
print(f"Columns: {list(unique_drugs_df.columns)}")
print(f"\nFirst 5 drugs:")
print(unique_drugs_df.head())

Loading drug data...
Loaded 1028 unique drugs from original dataset

Dataset shape: (1028, 2)
Columns: ['drug_chembl_id', 'smiles']

First 5 drugs:
  drug_chembl_id                                          smiles
0     CHEMBL1000     O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
1     CHEMBL1002               CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1
2     CHEMBL1004                  CN(C)CCOC(C)(c1ccccc1)c1ccccn1
3     CHEMBL1005  CCC(=O)N(c1ccccc1)C1(C(=O)OC)CCN(CCC(=O)OC)CC1
4     CHEMBL1009                  N[C@@H](Cc1ccc(O)c(O)c1)C(=O)O
Loaded 1028 unique drugs from original dataset

Dataset shape: (1028, 2)
Columns: ['drug_chembl_id', 'smiles']

First 5 drugs:
  drug_chembl_id                                          smiles
0     CHEMBL1000     O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
1     CHEMBL1002               CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1
2     CHEMBL1004                  CN(C)CCOC(C)(c1ccccc1)c1ccccn1
3     CHEMBL1005  CCC(=O)N(c1ccccc1)C1(C(=O)OC)CCN(CCC(=O)OC)CC1
4     CHEM

In [3]:
# Validate SMILES strings
print("Validating SMILES strings...")

# Check for missing SMILES
missing_smiles = unique_drugs_df['smiles'].isnull().sum()
print(f"Missing SMILES: {missing_smiles}")

# Check SMILES format (basic validation)
sample_smiles = unique_drugs_df['smiles'].iloc[0]
print(f"Sample SMILES: {sample_smiles}")
print(f"SMILES length range: {unique_drugs_df['smiles'].str.len().min()} - {unique_drugs_df['smiles'].str.len().max()}")

# Remove any rows with missing SMILES
if missing_smiles > 0:
    unique_drugs_df = unique_drugs_df.dropna(subset=['smiles'])
    print(f"Removed {missing_smiles} drugs with missing SMILES")
    print(f"Final dataset size: {len(unique_drugs_df)} drugs")

print(f"\nReady to encode {len(unique_drugs_df)} drugs")

Validating SMILES strings...
Missing SMILES: 0
Sample SMILES: O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
SMILES length range: 1 - 404

Ready to encode 1028 drugs


## 3. Initialize SMILES Encoder

In [4]:
# Define encoding functions
def smiles_to_mol(smiles):
    """Convert SMILES string to RDKit molecule object"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return mol
    except:
        return None

def get_morgan_fingerprint(mol, radius=2, nBits=2048):
    """Generate Morgan fingerprint (ECFP)"""
    if mol is None:
        return np.zeros(nBits)
    try:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
        return np.array(fp)
    except:
        return np.zeros(nBits)

def get_maccs_keys(mol):
    """Generate MACCS keys fingerprint"""
    if mol is None:
        return np.zeros(167)  # MACCS keys have 167 bits
    try:
        fp = MACCSkeys.GenMACCSKeys(mol)
        return np.array(fp)
    except:
        return np.zeros(167)

def get_molecular_descriptors(mol):
    """Calculate molecular descriptors"""
    if mol is None:
        return np.zeros(8)  # Return zeros for 8 descriptors
    
    try:
        descriptors = []
        descriptors.append(Descriptors.MolWt(mol))                    # Molecular weight
        descriptors.append(Descriptors.MolLogP(mol))                 # LogP
        descriptors.append(Descriptors.NumHDonors(mol))              # H-bond donors
        descriptors.append(Descriptors.NumHAcceptors(mol))           # H-bond acceptors
        descriptors.append(Descriptors.TPSA(mol))                    # Topological polar surface area
        descriptors.append(Descriptors.NumRotatableBonds(mol))       # Rotatable bonds
        descriptors.append(Descriptors.NumAromaticRings(mol))        # Aromatic rings
        descriptors.append(Descriptors.NumAliphaticRings(mol))       # Aliphatic rings
        
        return np.array(descriptors)
    except:
        return np.zeros(8)

print("SMILES encoding functions initialized!")
print("Available encoding methods:")
print("1. Morgan Fingerprints (ECFP): 2048-bit circular fingerprints")
print("2. MACCS Keys: 167-bit structural keys") 
print("3. Molecular Descriptors: 8 physicochemical properties")

SMILES encoding functions initialized!
Available encoding methods:
1. Morgan Fingerprints (ECFP): 2048-bit circular fingerprints
2. MACCS Keys: 167-bit structural keys
3. Molecular Descriptors: 8 physicochemical properties


## 4. Encode SMILES to Vectors

In [5]:
# Test encoding on a sample molecule first
print("Testing encoding functions on sample molecule...")
sample_smiles = unique_drugs_df['smiles'].iloc[0]
sample_mol = smiles_to_mol(sample_smiles)

if sample_mol is not None:
    morgan_fp = get_morgan_fingerprint(sample_mol)
    maccs_fp = get_maccs_keys(sample_mol)
    descriptors = get_molecular_descriptors(sample_mol)
    
    print(f"Sample SMILES: {sample_smiles}")
    print(f"Morgan fingerprint shape: {morgan_fp.shape}")
    print(f"MACCS keys shape: {maccs_fp.shape}")
    print(f"Molecular descriptors shape: {descriptors.shape}")
    print(f"Total feature vector size: {len(morgan_fp) + len(maccs_fp) + len(descriptors)}")
else:
    print("Failed to process sample molecule")
    
print("\n" + "="*60)

Testing encoding functions on sample molecule...
Sample SMILES: O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1
Morgan fingerprint shape: (2048,)
MACCS keys shape: (167,)
Molecular descriptors shape: (8,)
Total feature vector size: 2223



[02:36:44] DEPRECATION WARNING: please use MorganGenerator


In [6]:
# Process all SMILES strings and create encodings
print("Starting SMILES-to-vector encoding for all drugs...")
print(f"Processing {len(unique_drugs_df)} drugs...")

# Initialize lists to store encodings
morgan_fingerprints = []
maccs_fingerprints = []
molecular_descriptors_list = []
failed_smiles = []
successful_count = 0

# Process each SMILES string
for idx, row in unique_drugs_df.iterrows():
    drug_id = row['drug_chembl_id']
    smiles = row['smiles']
    
    # Convert SMILES to molecule
    mol = smiles_to_mol(smiles)
    
    if mol is not None:
        # Generate encodings
        morgan_fp = get_morgan_fingerprint(mol)
        maccs_fp = get_maccs_keys(mol)
        descriptors = get_molecular_descriptors(mol)
        
        morgan_fingerprints.append(morgan_fp)
        maccs_fingerprints.append(maccs_fp)
        molecular_descriptors_list.append(descriptors)
        successful_count += 1
    else:
        # Handle failed SMILES
        morgan_fingerprints.append(np.zeros(2048))
        maccs_fingerprints.append(np.zeros(167))
        molecular_descriptors_list.append(np.zeros(8))
        failed_smiles.append((drug_id, smiles))
    
    # Progress indicator
    if (idx + 1) % 100 == 0:
        print(f"Processed {idx + 1}/{len(unique_drugs_df)} drugs...")

print(f"\nEncoding completed!")
print(f"Successfully processed: {successful_count}/{len(unique_drugs_df)} drugs")
print(f"Failed SMILES: {len(failed_smiles)}")

if len(failed_smiles) > 0:
    print(f"Failed drugs (first 5): {failed_smiles[:5]}")

# Convert lists to numpy arrays
morgan_matrix = np.array(morgan_fingerprints)
maccs_matrix = np.array(maccs_fingerprints)
descriptors_matrix = np.array(molecular_descriptors_list)

print(f"\nEncoding matrices shapes:")
print(f"Morgan fingerprints: {morgan_matrix.shape}")
print(f"MACCS keys: {maccs_matrix.shape}")
print(f"Molecular descriptors: {descriptors_matrix.shape}")

Starting SMILES-to-vector encoding for all drugs...
Processing 1028 drugs...


[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerator
[02:36:54] DEPRECATION WARNING: please use MorganGenerat

Processed 100/1028 drugs...


[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerator
[02:36:55] DEPRECATION WARNING: please use MorganGenerat

Processed 200/1028 drugs...


...

[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerator
[02:37:01] DEPRECATION WARNING: please use MorganGenerat

Processed 900/1028 drugs...


[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerator
[02:37:02] DEPRECATION WARNING: please use MorganGenerat

Processed 1000/1028 drugs...

Encoding completed!
Successfully processed: 1028/1028 drugs
Failed SMILES: 0

Encoding matrices shapes:
Morgan fingerprints: (1028, 2048)
MACCS keys: (1028, 167)
Molecular descriptors: (1028, 8)

Encoding matrices shapes:
Morgan fingerprints: (1028, 2048)
MACCS keys: (1028, 167)
Molecular descriptors: (1028, 8)


[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator
[02:37:03] DEPRECATION WARNING: please use MorganGenerator


## 5. Create DataFrame with Encoded Data

In [7]:
# Create the encoded DataFrame
print("Creating DataFrame with encoded features...")

# Start with drug identifiers and SMILES
encoded_df = unique_drugs_df[['drug_chembl_id', 'smiles']].copy()

# Add Morgan fingerprint columns
morgan_columns = [f'morgan_{i:04d}' for i in range(2048)]
morgan_df = pd.DataFrame(morgan_matrix, columns=morgan_columns, index=encoded_df.index)

# Add MACCS keys columns
maccs_columns = [f'maccs_{i:03d}' for i in range(167)]
maccs_df = pd.DataFrame(maccs_matrix, columns=maccs_columns, index=encoded_df.index)

# Add molecular descriptor columns
descriptor_names = ['mol_weight', 'logp', 'h_donors', 'h_acceptors', 
                   'tpsa', 'rotatable_bonds', 'aromatic_rings', 'aliphatic_rings']
descriptors_df = pd.DataFrame(descriptors_matrix, columns=descriptor_names, index=encoded_df.index)

# Combine all features
print("Combining all feature types...")
encoded_df = pd.concat([encoded_df, morgan_df, maccs_df, descriptors_df], axis=1)

print(f"Final encoded DataFrame created!")
print(f"Shape: {encoded_df.shape}")
print(f"Columns: Drug ID, SMILES, {len(morgan_columns)} Morgan features, {len(maccs_columns)} MACCS features, {len(descriptor_names)} descriptors")

# Display memory usage
memory_usage = encoded_df.memory_usage(deep=True).sum() / (1024**2)
print(f"Memory usage: {memory_usage:.2f} MB")

# Show sample of the data
print(f"\nSample of encoded data (showing first few columns of each type):")
sample_cols = ['drug_chembl_id', 'smiles'] + morgan_columns[:3] + maccs_columns[:3] + descriptor_names[:3]
print(encoded_df[sample_cols].head(3))

Creating DataFrame with encoded features...
Combining all feature types...
Final encoded DataFrame created!
Shape: (1028, 2225)
Columns: Drug ID, SMILES, 2048 Morgan features, 167 MACCS features, 8 descriptors
Memory usage: 17.60 MB

Sample of encoded data (showing first few columns of each type):
  drug_chembl_id                                       smiles  morgan_0000  \
0     CHEMBL1000  O=C(O)COCCN1CCN(C(c2ccccc2)c2ccc(Cl)cc2)CC1            0   
1     CHEMBL1002            CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1            0   
2     CHEMBL1004               CN(C)CCOC(C)(c1ccccc1)c1ccccn1            0   

   morgan_0001  morgan_0002  maccs_000  maccs_001  maccs_002  mol_weight  \
0            1            0          0          0          0     388.895   
1            1            0          0          0          0     239.315   
2            0            0          0          0          0     270.376   

     logp  h_donors  
0  3.1482       1.0  
1  1.3060       4.0  
2  2.9233       0

In [8]:
# Analyze the molecular descriptors
print("Molecular Descriptors Analysis:")
print("=" * 50)

descriptors_stats = descriptors_df.describe()
print(descriptors_stats)

# Check for any NaN or infinite values
nan_counts = descriptors_df.isnull().sum()
inf_counts = np.isinf(descriptors_df.select_dtypes(include=np.number)).sum()

print(f"\nData Quality Check:")
print(f"NaN values: {nan_counts.sum()}")
print(f"Infinite values: {inf_counts.sum()}")

if nan_counts.sum() > 0:
    print("Columns with NaN values:", nan_counts[nan_counts > 0].index.tolist())
    
if inf_counts.sum() > 0:
    print("Columns with infinite values:", inf_counts[inf_counts > 0].index.tolist())

# Feature sparsity analysis for fingerprints
morgan_sparsity = (morgan_matrix == 0).mean() * 100
maccs_sparsity = (maccs_matrix == 0).mean() * 100

print(f"\nFeature Sparsity:")
print(f"Morgan fingerprints: {morgan_sparsity:.1f}% zeros")
print(f"MACCS keys: {maccs_sparsity:.1f}% zeros")

Molecular Descriptors Analysis:
        mol_weight         logp     h_donors  h_acceptors         tpsa  \
count  1028.000000  1028.000000  1028.000000  1028.000000  1028.000000   
mean    393.011962     2.421629     2.225681     5.433852    93.594825   
std     219.397452     2.881746     2.859182     3.920399    83.428354   
min      17.031000   -16.212860     0.000000     0.000000     0.000000   
25%     266.068500     1.147015     1.000000     3.000000    50.720000   
50%     362.848000     2.665070     2.000000     5.000000    79.455000   
75%     457.926750     4.055865     3.000000     7.000000   108.745000   
max    2639.187000    17.853900    42.000000    45.000000  1133.840000   

       rotatable_bonds  aromatic_rings  aliphatic_rings  
count      1028.000000     1028.000000      1028.000000  
mean          5.618677        1.716926         1.351167  
std           4.950241        1.307839         1.526854  
min           0.000000        0.000000         0.000000  
25%        

## 6. Save Encoded Data to Parquet

In [9]:
# Save the encoded dataset
output_filename = 'encoded_drugs_smiles_to_vector.parquet'

print("Saving encoded dataset...")
encoded_df.to_parquet(output_filename, index=False)

# Get file size
file_size = os.path.getsize(output_filename) / (1024**2)

print(f"Successfully saved to: {output_filename}")
print(f"File size: {file_size:.2f} MB")

# Final summary
print(f"\nSMILES-to-Vector Encoding Complete!")
print("=" * 60)
print(f"Dataset Summary:")
print(f"   • Total drugs encoded: {len(encoded_df)}")
print(f"   • Feature vector dimensions: {encoded_df.shape[1] - 2}")  # Exclude drug_id and smiles
print(f"   • Morgan fingerprints: 2048 features")
print(f"   • MACCS keys: 167 features") 
print(f"   • Molecular descriptors: 8 features")
print(f"   • Total molecular features: {2048 + 167 + 8}")
print(f"   • Successfully processed: {successful_count}/{len(unique_drugs_df)} drugs")
print(f"   • Output file: {output_filename}")

print(f"\nReady for machine learning!")
print("The encoded dataset contains comprehensive molecular representations suitable for:")
print("• Drug-protein interaction prediction")
print("• Molecular property prediction") 
print("• Chemical similarity analysis")
print("• Structure-activity relationship modeling")

Saving encoded dataset...
Successfully saved to: encoded_drugs_smiles_to_vector.parquet
File size: 1.44 MB

SMILES-to-Vector Encoding Complete!
Dataset Summary:
   • Total drugs encoded: 1028
   • Feature vector dimensions: 2223
   • Morgan fingerprints: 2048 features
   • MACCS keys: 167 features
   • Molecular descriptors: 8 features
   • Total molecular features: 2223
   • Successfully processed: 1028/1028 drugs
   • Output file: encoded_drugs_smiles_to_vector.parquet

Ready for machine learning!
The encoded dataset contains comprehensive molecular representations suitable for:
• Drug-protein interaction prediction
• Molecular property prediction
• Chemical similarity analysis
• Structure-activity relationship modeling
Successfully saved to: encoded_drugs_smiles_to_vector.parquet
File size: 1.44 MB

SMILES-to-Vector Encoding Complete!
Dataset Summary:
   • Total drugs encoded: 1028
   • Feature vector dimensions: 2223
   • Morgan fingerprints: 2048 features
   • MACCS keys: 167 feat

## Feature Comparison: One-Hot vs SMILES-to-Vector

| Encoding Method | Dimensions | Type | Interpretability | Information Content |
|----------------|------------|------|------------------|-------------------|
| **One-Hot** | 1,028 | Sparse Binary | High (drug identity) | Drug identity only |
| **SMILES-to-Vector** | 2,223 | Dense Numerical | Medium (molecular features) | Rich molecular properties |

### SMILES-to-Vector Features Breakdown:
- **Morgan Fingerprints (2048)**: Capture local molecular environments and structural patterns
- **MACCS Keys (167)**: Standard structural alerts and pharmacophore patterns  
- **Molecular Descriptors (8)**: Physicochemical properties (MW, LogP, TPSA, etc.)

### Next Steps:
1. **Compare Performance**: Train models using both encoding methods
2. **Feature Engineering**: Combine both approaches for hybrid models
3. **Dimensionality Reduction**: Apply PCA/t-SNE for visualization
4. **Model Training**: Use for drug-protein interaction prediction